# Personnaliser so modele

Faire sa propre fonction de perte

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
y_train,y_test = keras.utils.to_categorical(y_train,10),keras.utils.to_categorical(y_test,10)
print(y_train.shape)

(60000, 10)


In [4]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = X_train.shape[1:]),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(10,activation="softmax")
])

In [5]:
def huber_fn(y_true,y_pred):
    error = tf.cast(y_true, tf.float32)-y_pred
    is_small_error = tf.abs(error)<1
    squarred_loss = tf.square(error)/2
    linear_loss = tf.abs(error)-0.5
    return tf.where(is_small_error,x=squarred_loss,y=linear_loss)


In [6]:
optimizer = keras.optimizers.Nadam(learning_rate=0.001)
model.compile(loss=huber_fn,optimizer=optimizer,metrics=["accuracy"])


In [ ]:
lr = keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
cp = keras.callbacks.ModelCheckpoint(filepath="model_custom_loss",
                                     save_best_only=True,
                                     overwrite = True,
                                     save_weights_only= False)
model.fit(X_train,y_train,validation_split=0.2,epochs=10,callbacks = [lr,cp],verbose=1)

Epoch 1/10
1498/1500 [============================>.] - ETA: 0s - loss: 0.0290 - accuracy: 0.7099INFO:tensorflow:Assets written to: model_custom_loss\assets


INFO:tensorflow:Assets written to: model_custom_loss\assets


1500/1500 [==============================] - 16s 10ms/step - loss: 0.0290 - accuracy: 0.7099 - val_loss: 0.0326 - val_accuracy: 0.6743 - lr: 0.0010
Epoch 2/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0320 - accuracy: 0.6804 - val_loss: 0.0410 - val_accuracy: 0.5903 - lr: 9.0000e-04
Epoch 3/10
1497/1500 [============================>.] - ETA: 0s - loss: 0.0379 - accuracy: 0.6205INFO:tensorflow:Assets written to: model_custom_loss\assets


INFO:tensorflow:Assets written to: model_custom_loss\assets


1500/1500 [==============================] - 15s 10ms/step - loss: 0.0379 - accuracy: 0.6207 - val_loss: 0.0278 - val_accuracy: 0.7221 - lr: 8.1000e-04
Epoch 4/10
1500/1500 [==============================] - 11s 8ms/step - loss: 0.0267 - accuracy: 0.7327 - val_loss: 0.0335 - val_accuracy: 0.6653 - lr: 7.2900e-04
Epoch 5/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0325 - accuracy: 0.6751 - val_loss: 0.0311 - val_accuracy: 0.6889 - lr: 6.5610e-04
Epoch 6/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0336 - accuracy: 0.6641 - val_loss: 0.0302 - val_accuracy: 0.6976 - lr: 5.9049e-04
Epoch 7/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0339 - accuracy: 0.6611 - val_loss: 0.0286 - val_accuracy: 0.7142 - lr: 5.3144e-04
Epoch 8/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0317 - accuracy: 0.6830 - val_loss: 0.0339 - val_accuracy: 0.6612 - lr: 4.7830e-04
Epoch 9/10
1500/1500 [================

In [25]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 2s 5ms/step - loss: 0.0336 - accuracy: 0.6637


[0.03362317010760307, 0.6636999845504761]

Charger le model avec un élément personnalisé

In [26]:
model_loaded = keras.models.load_model("model_custom_loss",
                                       custom_objects={"huber_fn":huber_fn})

Si on veut pouvoir mettre un paramètre à notre fonction:

In [27]:
def create_huber_fn(threshold=1):
    def huber_fn(y_true,y_pred):
        error = tf.cast(y_true, tf.float32)-y_pred
        is_small_error = tf.abs(error)<threshold
        squarred_loss = tf.square(error)/2
        linear_loss = threshold*tf.abs(error)-threshold**2/2
        return tf.where(is_small_error,x=squarred_loss,y=linear_loss)
    return huber_fn


In [ ]:
optimizer = keras.optimizers.Nadam(learning_rate=0.001)
model.compile(loss=create_huber_fn(2),optimizer=optimizer,metrics=["accuracy"])
model_loaded = keras.models.load_model("model_custom_loss",
                                       custom_objects={create_huber_fn(2)})

Pour garder le threshhold enregistré lorsqu'on charge le modèle sans avoir à spécifier sa valeur quand on le load

In [ ]:
class HuberLoss(keras.losses.Loss):

    def __init__(self, threshold=1,**kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold

    def call(self,y_true,y_pred): # call pour losses, layers et models
        error = tf.cast(y_true, tf.float32)-y_pred
        is_small_error = tf.abs(error)<self.threshold
        squarred_loss = tf.square(error)/2
        linear_loss = self.threshold*tf.abs(error)-self.threshold**2/2
        return tf.where(is_small_error,x=squarred_loss,y=linear_loss)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config,"threshold":self.threshold}

In [ ]:
optimizer = keras.optimizers.Nadam(learning_rate=0.001)
model.compile(loss=HuberLoss(2),optimizer=optimizer,metrics=["accuracy"])
model_loaded = keras.models.load_model("model_custom_loss",
                                       custom_objects={HuberLoss})

# Faire son propre régulariseur (ici équivalent à "l1") et sa propre contrainte

Régulariseur (pour ne pas avoir à spécifier la valeur du facteur de régularisation à chaque fois qu'on load le model on fait hériter de la class keras)

In [ ]:
class MyReguliser(keras.regularizers.Regularizer):

    def __init__(self,factor):
        super().__init__()
        self.factor = factor

    def __call__(self, weights): # __call__ pour regularizers, initializers et constraints
        return tf.reduce_sum(tf.abs(self.factor*weights))
    
    def get_config(self):
        return {**super().get_config(), "factor":self.factor}

In [8]:
def my_constraint(weights):
    return tf.where(weights<0,tf.zeros_like(weights),weights)

In [9]:
layer = keras.layers.Dense(30,
                           kernel_initializer="he_normal",
                           kernel_constraint=my_constraint,
                           kernel_regularizer=MyReguliser)

On peut toujours faire la même chose: créer une classe qui hérite de keras en spécifiant la config pour ne pas avoir de pb quand on load le model et modifier la méthode __call__ pour regularizers, initializers et constraints et call pour loss, layers et models

# Faire sa propre Layer

## Sans paramètres (ex: flatten ou activation)

In [ ]:
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

## Avec paramètres

In [ ]:
class MyLayer(keras.layers.Layer):
    
    def __init__(self,units,activation=None,**kwargs):
        super().__init__(**kwargs)
        self.units = units #nb de neurones dans la couche dense
        self.activation = keras.activations.get(activation)
    
    def build(self, batch_input_shape):  # activé la première fois qu'on charge le modèle
        self.kernel=self.add_weight(name="kernel",shape=[batch_input_shape[-1],self.units],initializer="golrot_normal")
        self.bias = self.add_weight(name="bias",shape=[self.units],initializer="zeros")

    def call(self,X):  # appelé pour connaître la sortie de la couche
        return X @ self.kernel + self.bias
    
    def get_config(self): # utile quand on veut load le modèle sans passer les units et activation en paramètre: il les concervera automatiquement
        base_config = super().get_config()
        return {**base_config,"units":self.units,"activation":keras.activations.serialize(self.activation)}
        